# Movie Recommendation System using Content-Based Filtering

![alt text](recommendation-filtering-types.png "Title")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

movies_df = pd.read_csv("kaggle/input/the-movies-dataset/movies_metadata.csv")
ratings_df = pd.read_csv("kaggle/input/the-movies-dataset/ratings.csv")

In [ ]:
print(movies_df.shape)
print(movies_df.columns)

# Data cleaning

In [ ]:
movies_df.dropna(inplace=True)

In [ ]:
movies_df.duplicated().sum()

In [ ]:
movies_df.drop_duplicates(inplace=True)

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

In [ ]:
ratings_df.columns

In [ ]:
len(ratings_df)

# according to dataset description, only 9k of movies have ratings, we should merge them

In [ ]:
print("Movies dataframe datatypes:", movies_df.dtypes)
print("Ratings dataframe datatypes:", ratings_df.dtypes)

In [ ]:
movies_df["id"] = movies_df["id"].astype(int)
movies_ratings_df = movies_df.merge(right=ratings_df, 
                                           how="inner", left_on="id", right_on="movieId")

movies_ratings_df.dropna(inplace=True)
movies_ratings_df.drop_duplicates(subset="id", inplace=True)

# DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False,
#                 right_index=False, sort=False, suffixes=('_x', '_y'), 

movies_ratings_df.head()

In [ ]:
movies_ratings_df.columns

In [ ]:
movies_ratings_df.shape

In [ ]:
movies_ratings_df = movies_ratings_df.sample(frac=1).reset_index(drop=True)


In [ ]:
movies_ratings_df = movies_ratings_df.drop(
    movies_ratings_df[(movies_ratings_df["id"] == np.nan) | (movies_ratings_df["movieId"] == np.nan)].index)

In [ ]:
movies_ratings_df.shape

# need to extract genre

In [ ]:
import ast

def extract_genre(genre_str):
    try:
        genres = ast.literal_eval(genre_str)  # Convert string to list
        if isinstance(genres, list) and genres:
            for genre in genres:
                if "name" in genre:
                    return genre["name"]
    except (ValueError, SyntaxError):
        pass
    return ""


In [ ]:
collection_df = movies_ratings_df[["movieId","title", "genres","vote_average", "rating"]].copy()

In [ ]:
collection_df["genres"] = collection_df["genres"].apply(extract_genre)
# df['Book-Author'] = df['Book-Author'].apply(clean_text)

In [ ]:
collection_df.head()

In [ ]:
collection_df.shape

## Creating TfidVectorizer to vectorize the title and the genre ("CountVectorizer simply counts word occurrences, while TfidfVectorizer weighs words based on their importance within a document and across the entire corpus, giving more weight to rare and important words"). -> Embeddings

## cosine similarity measures the similarity between word vector embeddings within the vector space based on the angle and if they point in the same direction, indicating similarity

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def text_preprocess(text):
    return text.lower()


vectorizer = TfidfVectorizer(preprocessor=text_preprocess)

X = vectorizer.fit_transform(collection_df["title"] + " " + collection_df["genres"])

## recommendation test

In [ ]:
LIMIT = 5

print("ENTER INTERESTS: horror, adventure, action, crime, family, thriller, comedy, fantasy, animation, documentary, drama, science fiction, western, mystery")
print()

user_query = "action"

query = vectorizer.transform([user_query])

results = cosine_similarity(X, query).reshape((-1,))

print("Interests: ", query)
print("Recommendations: ")

for i in results.argsort()[-LIMIT:][::-1]:
    print(collection_df.iloc[i]["title"] + " : " + collection_df.iloc[i]["genres"])
    print("\n\n")

In [ ]:
print("ENTER INTERESTS: horror, adventure, action, crime, family, thriller, comedy, fantasy, animation, documentary, drama, science fiction, western, mystery")
print()

user_query = "family"

query = vectorizer.transform([user_query])

results = cosine_similarity(X, query).reshape((-1,))

print("Interests: ", query)
print("Recommendations: ")

for i in results.argsort()[-LIMIT:][::-1]:
    print(collection_df.iloc[i]["title"] + " : " + collection_df.iloc[i]["genres"])
    print("\n\n")